In [1]:
from google.colab import drive
drive._mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf

In [3]:
import os
import tensorflow_datasets as tfds

In [4]:
# !pip install -q git+https://github.com/tensorflow/docs

In [5]:
# from tensorflow_docs.vis import embed
from tensorflow.keras import layers
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import imageio
import cv2
import gc
from sklearn.metrics import accuracy_score
import time
import os
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [6]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

In [7]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [8]:
csv=pd.read_csv("/content/drive/MyDrive/Transformer_1/allCsv_2.csv")

In [9]:
from sklearn.model_selection import train_test_split
trainCsv=pd.DataFrame()
valCsv=pd.DataFrame()
n_of_each_class=30
n_classes=len((csv['classLabel'].unique()))
for Class in sorted((csv['classLabel'].unique())):
  t,v=train_test_split(csv[csv['classLabel']==Class].sample(n_of_each_class),test_size=0.25)
  trainCsv=pd.concat((trainCsv,t))
  valCsv=pd.concat((valCsv,v))

In [10]:
frameSize=(100,100)
nFrames=25
batch_size=16
params = {'batch_size': batch_size,
          'frameSize': frameSize,
          'nFrames': nFrames,
          'nclasses': n_classes,
          'oneHotEncode': True,
          'shuffle': True,
          'normalize': True,
          'std':False}

In [11]:
!pip install --upgrade ipyparallel
!ipcluster start -n 8

2022-01-03 18:08:41.241 [IPClusterStart] Using existing profile dir: '/root/.ipython/profile_default'
2022-01-03 18:08:41.407 [IPClusterStart] CRITICAL | Cluster is already running at /root/.ipython/profile_default/security/cluster-.json. use `ipcluster stop` to stop the cluster.


In [12]:
from ipyparallel import Client
with tf.device("/device:GPU:0"): 
  cli = Client()
  dview = cli[:]
  @dview.parallel(block=True)
  def loadData(i,index,listPaths,labels,nFrames,frameSize):
    import numpy as np
    import cv2,gc
    path = listPaths[index]
    try:
      vidcap=cv2.VideoCapture(path)
      totalFrames=int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
      focusFrames=np.linspace(0,totalFrames-1,nFrames).astype(int)
      success=True
      imgsBundle=np.empty((nFrames,frameSize[0],frameSize[1],3))
      i1=0
      countPush=0
      while success==True:
        success,image = vidcap.read()
        if i1 in focusFrames:
          img=cv2.resize(image,frameSize)
          imgsBundle[countPush]=img
          countPush+=1
        i1+=1
      gc.enable()
      gc.collect()
      labels1=labels[index]
      return imgsBundle,labels1
    except:
      imgsBundle=np.zeros((nFrames,frameSize[0],frameSize[1],3))
      labels1=0
      gc.enable()
      gc.collect()
      print("Weird data")
      return imgsBundle,labels1

In [13]:
# with strategy.scope():
with tf.device("/device:GPU:0"): 
  
  class Dataset(keras.utils.Sequence):
    'Characterizes a dataset for PyTorch'
    def __init__(self, listPaths, labels,batch_size=10,nFrames=30,frameSize=(300,300),nclasses=2,shuffle=True,oneHotEncode=False,normalize=True,std=False):
          'Initialization'
          self.labels = labels
          self.listPaths = listPaths
          self.nFrames = nFrames
          self.frameSize = frameSize
          self.oneHotEncode = oneHotEncode
          self.nclasses=nclasses
          self.shuffle=shuffle
          self.std=std
          self.normalize=normalize
          self.batch_size = batch_size
          self.on_epoch_start()
          # print('Data len',len(self.listPaths))

    def __len__(self):
      'Denotes the total number of samples'
      return int(np.floor(len(self.labels) / self.batch_size))

    def __getitem__(self,index):
      # print('index:',index)
      indexes = self.indexes[(index*self.batch_size):((index+1)*self.batch_size)]
      Xx,Yx=self.__data_generation(indexes)
      return Xx,Yx
      
    def on_epoch_start(self):
      'Updates indexes after each epoch'
      self.indexes = np.arange(len(self.labels))
      if self.shuffle == True:
          np.random.shuffle(self.indexes)
    def __data_generation(self, indexes):
      'Generates one sample of data'
      # data=loadData.map([{i,index,self.listPaths,self.labels,self.nFrames,self.frameSize} for i,index in enumerate(indexes)])
      data=loadData.map(range(len(indexes)),indexes,[self.listPaths]*len(indexes),[self.labels]*len(indexes),[self.nFrames]*len(indexes),[self.frameSize]*len(indexes))
      Y=np.asarray(data)[:,1]
      X=np.stack(np.asarray(data)[:,0],axis=0)
      if self.oneHotEncode==True:
        Y=keras.utils.to_categorical(Y,self.nclasses)
      if self.std==True and np.std(X)!=0:
        X=(X-np.mean(X))/np.std(X)
      if self.normalize==True:
        X=X/255
      # print("Shape of output by data generator: ",np.shape(X),' ',np.shape(Y))
      return X,Y

In [14]:
with tf.device("/device:GPU:0"): 
  training_generator = Dataset(trainCsv['Path'].values, trainCsv['classLabel'].values, **params)
  val_generator = Dataset(valCsv['Path'].values, valCsv['classLabel'].values, **params)

In [84]:
with tf.device("/device:GPU:0"): 
  class PositionalEmbedding(layers.Layer):
      def __init__(self, sequence_length, output_dim, **kwargs):
          super().__init__(**kwargs)
          self.position_embeddings = layers.Embedding(
              input_dim=sequence_length, output_dim=output_dim
          )
          self.sequence_length = sequence_length
          self.output_dim = output_dim

      def call(self, inputs):
          length = tf.shape(inputs)[1]
          positions = tf.range(start=0, limit=length, delta=1)
          embedded_positions = self.position_embeddings(positions)
          return inputs + embedded_positions

      def compute_mask(self, inputs, mask=None):
          mask = tf.reduce_any(tf.cast(inputs, "bool"), axis=-1)
          return mask

In [92]:
with tf.device("/device:GPU:0"): 
  class TransformerEncoder(layers.Layer):
      def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
          super().__init__(**kwargs)
          self.embed_dim = embed_dim
          self.dense_dim = dense_dim
          self.num_heads = num_heads
          self.attention = layers.MultiHeadAttention(
              num_heads=num_heads, key_dim=embed_dim, dropout=0.3
          )
          self.dense_proj = keras.Sequential(
              [layers.Dense(dense_dim, activation=tf.nn.gelu), layers.Dense(embed_dim),]
          )
          self.layernorm_1 = layers.LayerNormalization()
          self.layernorm_2 = layers.LayerNormalization()

      def call(self, inputs, mask=None):
          if mask is not None:
              mask = mask[:, tf.newaxis, :]

          attention_output = self.attention(inputs, inputs, attention_mask=mask)
          proj_input = self.layernorm_1(inputs + attention_output)
          proj_output = self.dense_proj(proj_input)
          return self.layernorm_2(proj_input + proj_output)

In [97]:
with tf.device("/device:GPU:0"): 
  def incModel(seqLen=20,embedDim=2048,numHeads=8,frameSize=(128,128),dense_reg=20,dense_dim = 4,dense_dim2=8):
    inputs = keras.Input(shape=(seqLen, frameSize[0], frameSize[1], 3))
    inc=InceptionV3(include_top=False, pooling='avg',input_shape=(frameSize[0],frameSize[1],3))
    x=layers.TimeDistributed(inc)(inputs)#out is 3D
    x=layers.Dense(dense_reg,activation='tanh')(x)
    print(np.shape(x))
    x = PositionalEmbedding(
        seqLen, np.shape(x)[-1], name="frame_position_embedding"
    )(x)
    x = TransformerEncoder(np.shape(x)[-1], dense_dim, numHeads, name="transformer_layer")(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(dense_dim2,activation='tanh')(x)
    model = keras.Model(inputs, x)
    return model

In [98]:
with tf.device("/device:GPU:0"): 
  def cnnModel(seqLen=20,embedDim=2048,numHeads=8,frameSize=(128,128),dense_dim = 4,dense_dim2=8):
    inputs = keras.Input(shape=(seqLen, frameSize[0], frameSize[1], 3))
    x=layers.TimeDistributed(layers.Conv2D(40,3,activation='tanh'))(inputs)
    x=layers.TimeDistributed(layers.MaxPooling2D())(x)
    x=layers.TimeDistributed(layers.Conv2D(20,3,activation='tanh'))(x)#x is 5D
    x=layers.TimeDistributed(layers.GlobalMaxPooling2D())(x)#x is 3D
    x = PositionalEmbedding(
        seqLen, np.shape(x)[-1], name="frame_position_embedding"
    )(x)
    print(np.shape(x))
    x = TransformerEncoder(np.shape(x)[-1], dense_dim, numHeads, name="transformer_layer")(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(dense_dim2,activation='tanh')(x)
    model = keras.Model(inputs, x)
    return model

In [101]:
with tf.device("/device:GPU:0"): 
  def mainModel(seqLen=nFrames,embedDim=2048,regDense=512,numHeads=32,frameSize=frameSize,classes=2,inc_dense_reg=20,dense_dim = 4,dense_dim2=8):
    cnn=cnnModel(seqLen=seqLen,embedDim=embedDim,numHeads=numHeads,frameSize=frameSize,dense_dim = dense_dim,dense_dim2=dense_dim2)
    inc=incModel(seqLen=seqLen,embedDim=embedDim,numHeads=numHeads,frameSize=frameSize,dense_dim = dense_dim,dense_dim2=dense_dim2,dense_reg=inc_dense_reg)
    inp=keras.Input(shape=(seqLen, frameSize[0], frameSize[1], 3))
    cnno=cnn(inp)
    print("cnno:",np.shape(cnno))
    inco=inc(inp)
    print("inco:",np.shape(inco))
    #----------------------MATHS SUM
    maino=tf.math.add(inco,cnno)
    #----------------------

    #----------------------CNN
    # maino=tf.stack((cnno,inco),axis=-1)
    # print("maino:",np.shape(maino))
    # maino=layers.Conv1D(10,2,activation='tanh')(maino)
    # print("maino:",np.shape(maino))
    # maino=layers.Flatten()(maino)
    #----------------------
    print("maino:",np.shape(maino))
    maino=layers.Dense(30,activation='tanh')(maino)
    maino=layers.Dense(20,activation='tanh')(maino)
    out=layers.Dense(classes,activation='softmax')(maino)
    model = keras.Model(inp, out)
    return model

In [102]:
with tf.device("/device:GPU:0"): 
  model=mainModel(embedDim=512,classes=14,dense_dim = 20,dense_dim2=15)

(None, 25, 20)
(None, 25, 20)
cnno: (None, 15)
inco: (None, 15)
maino: (None, 15)


In [103]:
with tf.device("/device:GPU:0"): 
# with strategy.scope(): 
  model.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
with tf.device("/device:GPU:0"): 
# with strategy.scope(): 
  epochs=100
  train_loss=[]
  val_loss=[]
  train_acc=[]
  val_acc=[]
  for epoch in range(epochs):
    start=time.time()
    print('Epoch: {}/{}'.format(epoch+1,epochs))
    count=0
    for i,data in enumerate(training_generator):
      x,y=data
      x=x.astype(np.float32)
      y=y.astype(np.float32)
      hist=model.fit(x,y,batch_size=batch_size,verbose=0,epochs=1)
      curr_acc=hist.history['accuracy']
      curr_loss=hist.history['loss']
      if count==0:
        train_loss.append(curr_loss[0])
        train_acc.append(curr_acc[0])
      else:
        train_loss[-1]+=curr_loss[0]
        train_acc[-1]+=curr_acc[0]
      count+=1
      gc.collect()
    train_loss[-1]/=count
    train_acc[-1]/=count
    count=0
    for i,data in enumerate(val_generator):
      x,y=data
      y_pred=model(x)
      curr_acc=accuracy_score(np.argmax(y,axis=1),np.argmax(y_pred,axis=1))
      curr_loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False,reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE)(y, y_pred).numpy()
      if count==0:
        val_loss.append(curr_loss)
        val_acc.append(curr_acc)
      else:
        val_loss[-1]+=curr_loss
        val_acc[-1]+=curr_acc
      count+=1
      gc.collect()
    val_loss[-1]/=count
    val_acc[-1]/=count
    end=time.time()
    gc.collect()
    print("Time:{} Training loss:{} Training accuracy:{} Validation loss:{} Validation accuracy:{}".format(end-start,train_loss[-1],train_acc[-1],val_loss[-1],val_acc[-1]))  

Epoch: 1/100
Time:132.40575671195984 Training loss:2.7743513333170036 Training accuracy:0.07894736842105263 Validation loss:2.7589239392961775 Validation accuracy:0.0625
Epoch: 2/100
Time:100.68751001358032 Training loss:2.6734635955409 Training accuracy:0.09868421052631579 Validation loss:2.7241499764578685 Validation accuracy:0.07142857142857142
Epoch: 3/100
Time:107.12449884414673 Training loss:2.5998346429122123 Training accuracy:0.09539473684210527 Validation loss:2.669249943324498 Validation accuracy:0.08928571428571429
Epoch: 4/100
Time:107.88843607902527 Training loss:2.547112540194863 Training accuracy:0.11842105263157894 Validation loss:2.637353079659598 Validation accuracy:0.08035714285714286
Epoch: 5/100
Time:90.27152800559998 Training loss:2.4555978273090564 Training accuracy:0.1513157894736842 Validation loss:2.5408382415771484 Validation accuracy:0.125
Epoch: 6/100
Time:83.1530487537384 Training loss:2.3487187435752466 Training accuracy:0.20723684210526316 Validation los

In [ ]:
#this seems fast 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_acc)
plt.plot(val_acc)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
plt.plot(train_loss)
plt.plot(val_loss)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()